In [9]:
import plotly.graph_objs as go
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import getpass
from datetime import datetime
from sklearn.metrics import r2_score


def plotly_voorsp_obs(act,
                      preds,
                      savepath,
                      horizons,
                      line_colors=["navy", "grey", "blue",
                                   "purple", "brown"],
                      quant_preds=None,
                      low_quant=0.1,
                      index=None,
                      save_figure=True,
                      title="Voorspelling en meting debiet",
                      yaxis_title="Debiet (m<sup>3</sup>/u)"):
    """Maak een plotly plot van voorspellingen en observaties.


    Parameters
    ----------
    act : list, numpy.array or pd.Series
        Debietobservaties.
    preds : list van lists, numpy.arrays or pd.Series
        Debietvoorspellingen.
    savepath : str
        Path waar de figuur gesaved moet worden.
    horizons : list of strs
        Lijst met voorspelhorizonnen
    line_colors: list of strs
        Lijst met kleuren lijnen, default value =
        ["navy", "grey", "blue", "purple", "brown"]
    quant_preds : list van 2 lists, numpy.arrays or pd.Series, optional
        Quantile predicties. The default is None.
    low_quant : float, optional
        Welke lower quantile is gebruikt? The default is 0.1.
    index : pd.index object, optional
        Index voor op de x-as. The default is None.
    save_figure: bool, optional
        Whether or not to save the figure. Default value is True.
    title : str, optional
        Titel grafiek. The default is "Voorspelling en meting".

    Raises
    ------
    ValueError
        If one of the lengths is not the same as the others.

    Returns
    -------
    fig : plotly figure object
    """

    # Create index
    if index is None:
        if type(act) == pd.Series:
            index = act.index
        else:
            if type(preds[0]) == pd.Series:
                index = preds[0].index
            else:
                raise ValueError(
                    "Either index should be given or act or pred should have an index.")

    # Test lengths
    if len(index) != len(act):
        raise ValueError("length index should be equal to length act")

    if len(index) != len(preds[0]):
        raise ValueError("length index should be equal to length pred")

    # Maak figuur
    fig = go.Figure()

    # Quantiles
    if quant_preds is not None:
        if len(index) != len(quant_preds[0]):
            raise ValueError("length index should be equal to length quants_preds[0]")
        if len(index) != len(quant_preds[1]):
            raise ValueError("length index should be equal to length quants_preds[1]")
        fig.add_trace(go.Scatter(x=index,
                                 y=quant_preds[0],
                                 fill=None,
                                 mode='lines',
                                 line_color='rgba(100,100,100,0.4)',
                                 name=f'{round(low_quant*100,1)}e percentiel',
                                 opacity=0.2
                                 ))

        fig.add_trace(go.Scatter(
            x=index,
            y=quant_preds[1],
            fill='tonexty',  # fill area between trace0 and trace1
            mode='lines', line_color='rgba(100,100,100,0.4)',
            opacity=0.2,
            fillcolor='rgba(100,100,100,0.2)',
            name=f'{round((1-low_quant)*100,1)}e percentiel'))

    # Observaties
    fig.add_trace(go.Scatter(go.Scatter(
        mode='lines+markers',
        x=index,
        y=act,
        name='Observatie',
        marker_symbol='diamond-open-dot',
        line_color='red',
        marker=dict(

            color='red',
            size=5,
            line=dict(
                color='red',
                width=1
            )))))

    # Voorspellingen
    for i, pred in enumerate(preds):
        fig.add_trace(go.Scatter(go.Scatter(
            x=index,
            y=pred,
            name=f'Voorsp. +{horizons[i]}',
            line_color=line_colors[i])))

    # Assentitels etc.
    fig.update_layout(
        title=title,
        xaxis_title="Datum",
        yaxis_title=yaxis_title,
        legend_title="Legenda",
        font=dict(
            family="Helvetica",
            size=14,
            color="grey"
        )
    )

    # Sla op als figuur
    if save_figure:
        fig.write_html(savepath,
                       include_plotlyjs=True)
    return fig

In [19]:
pred_exp_folder = fr"C:\Users\{getpass.getuser()}\OneDrive - Hoogheemraadschap van Delfland\3_Projecten\Zoutindringing\Data\voorspellingen\voorsp_2023_07_20"
predictions = pd.read_csv(fr"{pred_exp_folder}/voorspellingen_egv_parkhaven_light_all.csv").set_index('datetime')
measurements = pd.read_csv(fr"{pred_exp_folder}/metingen_egv_parkhaven_all.csv").set_index('datetime')

predictions.index = pd.to_datetime(predictions.index)
measurements.index = pd.to_datetime(measurements.index)

In [ ]:
time_periods = [
        [datetime(2015, 1, 1), datetime(2015, 2, 1)],
        [datetime(2016, 3, 1), datetime(2016, 4, 1)],
        [datetime(2017, 5, 1), datetime(2017, 6, 1)],
        [datetime(2018, 7, 1), datetime(2018, 8, 1)],
        [datetime(2019, 9, 1), datetime(2019, 10, 1)],
        [datetime(2020, 11, 1), datetime(2020, 12, 1)]
    ]

plot_horizons = predictions.columns

for time_period in time_periods:
    if len(measurements.loc[time_period[0]:time_period[1], 'EGV_OPP']) == 0:
        print(f"Time period {time_period} had no data")
        continue
    r2s = [r2_score(
        measurements.loc[time_period[0]:time_period[1], 'EGV_OPP'],
        predictions.loc[
            time_period[0]:time_period[1],
            horizon
        ]) for horizon in plot_horizons
    ]
    rerengai_out_plot = predictions.loc[time_period[0]:time_period[1], plot_horizons]
    plot_horizons_str = [
        f"{plot_horizon} R2: {round(r2, 3)}" for plot_horizon, r2 in zip(
            plot_horizons, r2s)
    ]
    a = r"C:\Users\SjoerdGnodde\OneDrive - Hoogheemraadschap van Delfland\3_Projecten\Zoutindringing\plots\parkhaven_pred_vs_meas\\"
    b = f"_{str(time_period[1].date())}.html"
    save_path_plotly = f"{a}{str(time_period[0].date())}{b}"

    plotly_voorsp_obs(
        measurements.loc[time_period[0]:time_period[1], 'target_0'],
        [rerengai_out_plot[col] for col in rerengai_out_plot],
        save_path_plotly,
        horizons=plot_horizons_str,
        line_colors=["navy", "grey", "blue",
                        "purple", "brown",
                        "indianred", "lightcoral"]
    )